In [1]:
import yasp
yasp.module_load('bundle/hepbase heppyy/current')

[i] Using yaspenv.sh at /Users/ploskon/devel/yasp/yaspenv.sh


In [2]:
import cppyy

import heppyy.util.fastjet_cppyy
import heppyy.util.pythia8_cppyy
import heppyy.util.heppyy_cppyy

from cppyy.gbl import fastjet as fj
from cppyy.gbl import Pythia8
from cppyy.gbl.std import vector

from heppyy.pythia_util import configuration as pyconf

import ROOT


/Users/ploskon/devel/yasp/software/bundle/hepbase/root/default/lib/cppyy_backend/loader.py:90: UserWarning: No precompiled header available (/Users/ploskon/devel/yasp/software/bundle/hepbase/root/default/lib/cppyy_backend/etc not writable); this may impact performance.
  warnings.warn('No precompiled header available (%s); this may impact performance.' % msg)
[i] Creating YaspCppyyHelper singleton.
[i] Including header: fastjet/PseudoJet.hh
[i] Including header: fastjet/JetDefinition.hh
[i] Including header: fastjet/ClusterSequence.hh
[i] Including header: fastjet/Selector.hh
[i] Including header: fastjet/contrib/LundGenerator.hh
[i] Including header: fastjet/contrib/Recluster.hh
[i] Including header: fastjet/contrib/SoftDrop.hh
[i] Including header: fastjet/contrib/EnergyCorrelator.hh
[i] Including header: Pythia8/Pythia.h


[i] cppyyhelper (4356732496)
   init_ignore_none = True
   args = None
   init_dict = None
   init_json = None
   init_yaml = None
   loaded_packages = ['fastjet']
   loaded_libs = ['fastjet', 'LundPlane', 'EnergyCorrelator']
   paths_include = ['/Users/ploskon/devel/yasp/software/fastjet/3.4.2/include']
   paths_lib = ['/Users/ploskon/devel/yasp/software/fastjet/3.4.2/lib']
[i] cppyyhelper (4356732496)
   init_ignore_none = True
   args = None
   init_dict = None
   init_json = None
   init_yaml = None
   loaded_packages = ['fastjet', 'pythia8']
   loaded_libs = ['fastjet', 'LundPlane', 'EnergyCorrelator', 'pythia8']
   paths_include = ['/Users/ploskon/devel/yasp/software/fastjet/3.4.2/include', '/Users/ploskon/devel/yasp/software/pythia8/default/include']
   paths_lib = ['/Users/ploskon/devel/yasp/software/fastjet/3.4.2/lib', '/Users/ploskon/devel/yasp/software/pythia8/default/lib']


[i] Including header: eec/ecorrel.hh
[i] Including header: fjutil/fjutil.hh
[i] Including header: fjext/fjtools.hh
[i] Including header: groom/GroomerShop.hh
[i] Including header: pythiaext/PythiaExt.hh
[i] Including header: hybridutil/readhybrid.hh
[i] Including header: hybridutil/hybridNegaRecombiner.hh


[i] cppyyhelper (4356732496)
   init_ignore_none = True
   args = None
   init_dict = None
   init_json = None
   init_yaml = None
   loaded_packages = ['fastjet', 'pythia8', 'heppyy']
   loaded_libs = ['fastjet', 'LundPlane', 'EnergyCorrelator', 'pythia8', 'heppyy_eec', 'heppyy_fjutil', 'heppyy_fjext', 'heppyy_groom', 'heppyy_pythiafjext', 'heppyy_pythiaext', 'heppyy_hybridutil']
   paths_include = ['/Users/ploskon/devel/yasp/software/fastjet/3.4.2/include', '/Users/ploskon/devel/yasp/software/pythia8/default/include', '/Users/ploskon/devel/yasp/software/heppyy/current/include']
   paths_lib = ['/Users/ploskon/devel/yasp/software/fastjet/3.4.2/lib', '/Users/ploskon/devel/yasp/software/pythia8/default/lib', '/Users/ploskon/devel/yasp/software/heppyy/current/lib']
Welcome to JupyROOT 6.30/06


In [3]:
pythia_config = [
    'HardQCD:all = on',
    'Next:numberCount = 0',
    'Next:numberShowEvent = 0',
    'Next:numberShowInfo = 0',
    'Next:numberShowProcess = 0',
    'PhaseSpace:pTHatMin = 100.00000'
]

stable_charm = False
if stable_charm:
	for c in [411,413,421,423,431,433]:
		pythia_config.append(f'{c}:mayDecay=false')
		pythia_config.append(f'-{c}:mayDecay=false')

stable_beauty = False
if stable_beauty:
	for c in [511,513,521,523,531,533]:
		pythia_config.append(f'{c}:mayDecay=false')
		pythia_config.append(f'-{c}:mayDecay=false')

pythia = Pythia8.Pythia()
_ = [pythia.readString(s) for s in pythia_config]
pythia.init()

fj.ClusterSequence.print_banner()
print()

# set up our jet definition and a jet selector
jet_R0 = 0.4
jet_def = fj.JetDefinition(fj.antikt_algorithm, jet_R0)
jet_selector = fj.SelectorPtMin(100.0) * fj.SelectorAbsEtaMax(1)
print(jet_def)

jet_def_lund = fj.JetDefinition(fj.cambridge_algorithm, 1.0)
lund_gen = fj.contrib.LundGenerator(jet_def_lund)
print('making lund diagram for all jets...')
print(f' {lund_gen.description()}')


making lund diagram for all jets...
 LundGenerator with Recluster with new_jet_def = Longitudinally invariant Cambridge/Aachen algorithm with R = 1 and E scheme recombination and keeping the hardest inclusive jet

 *------------------------------------------------------------------------------------* 
 |                                                                                    | 
 |  *------------------------------------------------------------------------------*  | 
 |  |                                                                              |  | 
 |  |                                                                              |  | 
 |  |   PPP   Y   Y  TTTTT  H   H  III    A      Welcome to the Lund Monte Carlo!  |  | 
 |  |   P  P   Y Y     T    H   H   I    A A     This is PYTHIA version 8.310      |  | 
 |  |   PPP     Y      T    HHHHH   I   AAAAA    Last date of change: 25 Jul 2023  |  | 
 |  |   P       Y      T    H   H   I   A   A                            

In [4]:
from tqdm.notebook import tqdm
nevents = 1
for i in tqdm(range(nevents)):
  if not pythia.next():
    continue
  parts = vector[fj.PseudoJet]([fj.PseudoJet(p.px(), p.py(), p.pz(), p.e()) for p in pythia.event if p.isFinal()])
  # parts = pythiafjext.vectorize(pythia, True, -1, 1, False)
  jets = jet_selector(jet_def(parts))
  for j in jets:
    lunds = lund_gen.result(j)
    print(f'jet pT={j.perp()}')
    for i, l in enumerate(lunds):
      print('- L {} pT={:5.2f} eta={:5.2f}'.format(i, l.pair().perp(), l.pair().eta()))
      print('  Deltas={}'.format(l.Delta()))
      print('  kts={}'.format(l.kt()))
      print()


pythia.stat()

print(type(pythia))


  0%|          | 0/1000 [00:00<?, ?it/s]

jet pT=183.92581029483244
- L 0 pT=183.93 eta= 0.37
  Deltas=0.38602531096925563
  kts=0.11879166499583851

- L 1 pT=183.63 eta= 0.37
  Deltas=0.34390115258488624
  kts=0.19082596273639776

- L 2 pT=183.10 eta= 0.37
  Deltas=0.2591895591283384
  kts=0.7679573262395816

- L 3 pT=180.18 eta= 0.36
  Deltas=0.1987299545586517
  kts=0.29406993067808024

- L 4 pT=178.73 eta= 0.36
  Deltas=0.16388003492570932
  kts=0.33462662793660686

- L 5 pT=176.70 eta= 0.36
  Deltas=0.10866777950866052
  kts=0.1748182019367301

- L 6 pT=175.10 eta= 0.36
  Deltas=0.07913842184070055
  kts=5.921570395219232

- L 7 pT=100.31 eta= 0.34
  Deltas=0.06086596125446628
  kts=1.2109575238131376

- L 8 pT=80.43 eta= 0.33
  Deltas=0.04143328069944637
  kts=0.7338437217101548

- L 9 pT=62.73 eta= 0.34
  Deltas=0.010986089515089833
  kts=0.1448380164312661

- L 10 pT=49.55 eta= 0.34
  Deltas=0.007830871811519112
  kts=0.0203207176115439

- L 11 pT=46.95 eta= 0.34
  Deltas=0.006536689060621268
  kts=0.0969431331602428

